**Use the evaluation_results.csv file as input for this code to find topsis scroe and rank the models**

In [3]:
evaluation_results = pd.read_csv("/kaggle/input/inputfortopsis/evaluation_results.csv")
print(evaluation_results)

                           Model  BLEU Score  Semantic Coherence  \
0        facebook/bart-large-cnn    0.348812            0.655736   
1                       t5-large    0.339788            0.755254   
2  sshleifer/distilbart-cnn-12-6    0.305259            0.708327   
3           google/pegasus-large    0.361715            0.707104   
4  allenai/led-large-16384-arxiv    0.152973            0.243868   

   Factual Accuracy  Content Coverage  
0          0.458349          0.371619  
1          0.457265          0.408908  
2          0.521721          0.483919  
3          0.420562          0.363004  
4          0.284099          0.186362  


**Import Dependencies**

In [8]:
# !pip install scikit-criteria
import numpy as np
import pandas as pd
from sklearn import preprocessing
from skcriteria import mkdm

**You may adjust the weights based on the importance of each criterion as per your choice**

Changing the weights may change the output

In [9]:
weights = [0.25, 0.25, 0.25, 0.25] 

**Normalize the scores and then calculate topsis scores and ranks for different models**

In [14]:
# Normalize the scores
normalized_scores = evaluation_results.iloc[:, 1:].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Convert the normalized scores to a numpy array
normalized_scores_array = normalized_scores.to_numpy()

# Define the ideal and negative-ideal solutions
ideal_solution = np.max(normalized_scores_array, axis=0)
negative_ideal_solution = np.min(normalized_scores_array, axis=0)

# Calculate the Euclidean distances to ideal and negative-ideal solutions
euclidean_distances_to_ideal = np.linalg.norm(normalized_scores_array - ideal_solution, axis=1)
euclidean_distances_to_negative_ideal = np.linalg.norm(normalized_scores_array - negative_ideal_solution, axis=1)

# Calculate the TOPSIS scores
topsis_scores = euclidean_distances_to_negative_ideal / (euclidean_distances_to_ideal + euclidean_distances_to_negative_ideal)

# Add TOPSIS scores to your evaluation_results DataFrame
evaluation_results["TOPSIS Score"] = topsis_scores

# Rank based on TOPSIS scores
evaluation_results["Rank"] = evaluation_results["TOPSIS Score"].rank(ascending=False)

# Display the final evaluation_results DataFrame
print(evaluation_results)

                           Model  BLEU Score  Semantic Coherence  \
0        facebook/bart-large-cnn    0.348812            0.655736   
1                       t5-large    0.339788            0.755254   
2  sshleifer/distilbart-cnn-12-6    0.305259            0.708327   
3           google/pegasus-large    0.361715            0.707104   
4  allenai/led-large-16384-arxiv    0.152973            0.243868   

   Factual Accuracy  Content Coverage  TOPSIS Score  Rank  
0          0.458349          0.371619      0.756144   3.0  
1          0.457265          0.408908      0.815415   2.0  
2          0.521721          0.483919      0.865146   1.0  
3          0.420562          0.363004      0.726354   4.0  
4          0.284099          0.186362      0.000000   5.0  


**Saving the output csv file**

In [15]:
evaluation_results.to_csv("topsis_results.csv", index=False)